In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import project

### Poses got from map -> camera_link, when in mapping mode (localization is False)

In [2]:
path_input = "../Results/Poses_Results/TF_no_localization/map_camera.txt"
path_output = "../Results/Poses_Results/TF_no_localization/map_camera_parsed.txt"

In [3]:
project.parse_pose_from_TF(path_input, path_output)

In [4]:
poses_TF_mapping = pd.read_csv(path_output)

In [5]:
poses_TF_mapping.head(5)

,Time,X,Y,Z,Roll,Pitch,Yaw
0,8603.967,0.890,-1.286,0.235,-0.0,-0.0,-2.156
1,8604.967,0.890,-1.286,0.235,-0.0,-0.0,-2.156
2,8605.968,0.890,-1.286,0.235,0.0,0.0,-2.156
3,8606.967,0.891,-1.286,0.235,-0.0,0.0,-2.156
4,8607.967,0.891,-1.286,0.235,-0.0,0.0,-2.156


In [6]:
poses_TF_mapping.shape

(195, 7)

### Poses got from map -> camera_link, when in localization mode (localization is True)

In [7]:
path_input = "../Results/Poses_Results/TF_with_localization/map_camera.txt"
path_output = "../Results/Poses_Results/TF_with_localization/map_camera_parsed.txt"

In [8]:
project.parse_pose_from_TF(path_input, path_output)

In [9]:
poses_TF_localization = pd.read_csv(path_output)

In [10]:
poses_TF_localization.head(5)

,Time,X,Y,Z,Roll,Pitch,Yaw
0,8603.967,0.89,-1.286,0.235,-0.0,-0.0,-2.156
1,8604.967,0.89,-1.286,0.235,-0.0,-0.0,-2.156
2,8605.968,0.89,-1.286,0.235,0.0,0.0,-2.157
3,8606.967,0.89,-1.287,0.235,-0.0,0.0,-2.156
4,8607.967,0.89,-1.288,0.235,-0.0,0.0,-2.156


In [11]:
poses_TF_localization.shape

(195, 7)

### Data from camera_info topic (no need to parse, it is always the same)

In [12]:
D = np.array([0.0, 0.0, 0.0, 0.0, 0.0])
K = np.array([554.3827128226441, 0.0, 320.5, 0.0, 554.3827128226441, 240.5, 0.0, 0.0, 1.0])

# Projection